<a href="https://colab.research.google.com/github/lorenzopaoria/Smoking-detection-and-distance-analysis/blob/main/model_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision.io import read_image
from torchvision.ops import nms
from torchvision.utils import draw_bounding_boxes
import os
from PIL import Image
from pathlib import Path

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
CLASS_NAMES = ["sigarette", "smoker", "non_smoker"]
NUM_CLASSES = len(CLASS_NAMES) + 1
MODEL_PATH = '/content/drive/MyDrive/pth_epoch/model_epoch_10.pth'
TEST_DIR = '/content/drive/MyDrive/Photo/test'
OUTPUT_DIR = '/content/drive/MyDrive/test_trained'

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [14]:
def get_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(
        in_features, NUM_CLASSES)

    return model

In [15]:
def load_model(model_path):
    """Carica il modello salvato"""
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Il file del modello non esiste: {model_path}")

    print(f"Caricamento del modello da {model_path}")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Utilizzo device: {device}")

    try:
        model = get_model()

        state_dict = torch.load(model_path, map_location=device)
        model.load_state_dict(state_dict)

        model.eval()
        print("Modello caricato con successo")
        return model, device
    except Exception as e:
        print(f"Errore nel caricamento del modello: {str(e)}")
        raise


In [16]:
class TestDataset(Dataset):
    def __init__(self, test_dir, transform=None):
        self.test_dir = Path(test_dir)
        if not self.test_dir.exists():
            raise FileNotFoundError(f"La directory {test_dir} non esiste!")

        self.image_files = list(self.test_dir.glob('*.[jp][pn][g]'))
        if not self.image_files:
            raise FileNotFoundError(f"Nessuna immagine trovata in {test_dir}")

        print(f"Trovate {len(self.image_files)} immagini in {test_dir}")

        self.transform = transform or transforms.Compose([
            transforms.Resize((800, 800)),  # Dimensione standard per Faster R-CNN
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                              std=[0.229, 0.224, 0.225])  # Normalizzazione ImageNet
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = str(self.image_files[idx])
        try:
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
            return image, img_path
        except Exception as e:
            print(f"Errore nel caricamento dell'immagine {img_path}: {str(e)}")
            raise

In [17]:
def draw_predictions(image, boxes, labels, scores, output_path):
    """Disegna le bounding box sulle immagini e salva il risultato"""
    try:
        # Denormalizza l'immagine
        image = image * torch.tensor([0.229, 0.224, 0.225])[:, None, None]
        image = image + torch.tensor([0.485, 0.456, 0.406])[:, None, None]
        image = (image * 255).byte()

        # Colori per le diverse classi
        colors = ["red", "green", "blue"]

        # Crea le etichette con classe e score
        labels_text = [f"{CLASS_NAMES[int(l)-1]}: {s:.2f}" for l, s in zip(labels, scores)]

        # Disegna le bounding box
        image_with_boxes = draw_bounding_boxes(
            image,
            boxes,
            labels=labels_text,
            colors=colors,
            width=2
        )

        # Converti in formato PIL e salva
        image_with_boxes = image_with_boxes.permute(1, 2, 0).numpy()
        plt.figure(figsize=(12, 8))
        plt.imshow(image_with_boxes)
        plt.axis('off')
        plt.savefig(output_path, bbox_inches='tight', pad_inches=0)
        plt.close()

        print(f"Salvata immagine con predizioni in {output_path}")
    except Exception as e:
        print(f"Errore nel disegno delle predizioni: {str(e)}")
        raise

In [18]:
def evaluate_model(model, device, test_loader):
    """Valuta il modello e salva le immagini con le predizioni"""
    model.to(device)

    print("Inizio valutazione del modello...")
    with torch.no_grad():
        for i, (images, img_paths) in enumerate(test_loader):
            print(f"\nProcessing batch {i+1}/{len(test_loader)}")

            try:
                images = images.to(device)
                predictions = model(images)

                for j, (prediction, img_path) in enumerate(zip(predictions, img_paths)):
                    boxes = prediction['boxes']
                    scores = prediction['scores']
                    labels = prediction['labels']

                    # Filtra le predizioni con score > 0.5
                    mask = scores > 0.5
                    boxes = boxes[mask]
                    scores = scores[mask]
                    labels = labels[mask]

                    if len(boxes) > 0:
                        base_name = os.path.basename(img_path)
                        output_filename = f"pred_{base_name}"
                        output_path = os.path.join(OUTPUT_DIR, output_filename)

                        draw_predictions(
                            images[j].cpu(),
                            boxes.cpu(),
                            labels.cpu(),
                            scores.cpu(),
                            output_path
                        )

                        print(f"Processata immagine: {base_name}")
                    else:
                        print(f"Nessuna predizione sopra la soglia per {os.path.basename(img_path)}")
            except Exception as e:
                print(f"Errore nel processing del batch {i+1}: {str(e)}")
                continue

In [19]:
if __name__ == "__main__":
    try:
        print("Inizializzazione...")

        # Crea la directory di output se non esiste
        os.makedirs(OUTPUT_DIR, exist_ok=True)

        # Verifica l'esistenza delle directory
        for path in [os.path.dirname(MODEL_PATH), TEST_DIR]:
            if not os.path.exists(path):
                raise FileNotFoundError(f"Directory non trovata: {path}")

        # Crea il dataset di test
        print("\nCreazione dataset di test...")
        test_dataset = TestDataset(TEST_DIR)
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

        # Carica il modello
        print("\nCaricamento modello...")
        model, device = load_model(MODEL_PATH)

        # Esegui la valutazione
        print("\nInizio valutazione...")
        evaluate_model(model, device, test_loader)

        print("\nProcessing completato con successo!")

    except Exception as e:
        print(f"\nErrore durante l'esecuzione: {str(e)}")

Inizializzazione...

Creazione dataset di test...
Trovate 73 immagini in /content/drive/MyDrive/Photo/test

Caricamento modello...
Caricamento del modello da /content/drive/MyDrive/pth_epoch/model_epoch_10.pth
Utilizzo device: cpu


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 58.8MB/s]
<ipython-input-15-4389a666b82e>:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.c

Errore nel caricamento del modello: Error(s) in loading state_dict for FasterRCNN:
	Missing key(s) in state_dict: "backbone.body.conv1.weight", "backbone.body.bn1.weight", "backbone.body.bn1.bias", "backbone.body.bn1.running_mean", "backbone.body.bn1.running_var", "backbone.body.layer1.0.conv1.weight", "backbone.body.layer1.0.bn1.weight", "backbone.body.layer1.0.bn1.bias", "backbone.body.layer1.0.bn1.running_mean", "backbone.body.layer1.0.bn1.running_var", "backbone.body.layer1.0.conv2.weight", "backbone.body.layer1.0.bn2.weight", "backbone.body.layer1.0.bn2.bias", "backbone.body.layer1.0.bn2.running_mean", "backbone.body.layer1.0.bn2.running_var", "backbone.body.layer1.0.conv3.weight", "backbone.body.layer1.0.bn3.weight", "backbone.body.layer1.0.bn3.bias", "backbone.body.layer1.0.bn3.running_mean", "backbone.body.layer1.0.bn3.running_var", "backbone.body.layer1.0.downsample.0.weight", "backbone.body.layer1.0.downsample.1.weight", "backbone.body.layer1.0.downsample.1.bias", "backbone.b